# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [2]:
len(text)

305189

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [4]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [4]:
view_sentence_range[1]

10

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [6]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    word_counts = Counter(text)
    #print(word_counts)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    #print(sorted_vocab)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    return vocab_to_int, int_to_vocab
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Counter({'moe_szyslak': 5, 'mike': 4, 'rotch': 4, 'you': 4, 'your': 4, 'to': 3, 'drink': 3, 'the': 2, 'yeah': 2, 'name': 2, 'on': 2, 'hey': 2, 'one': 2, "i'm": 2, 'gonna': 2, 'my': 2, 'homer': 2, 'not': 2, 'problems': 2, 'should': 2, "moe's": 1, 'tavern': 1, 'where': 1, 'elite': 1, 'meet': 1, 'bart_simpson': 1, 'eh': 1, 'hello': 1, 'is': 1, 'there': 1, 'last': 1, 'hold': 1, "i'll": 1, 'check': 1, 'has': 1, 'anybody': 1, 'seen': 1, 'lately': 1, 'listen': 1, 'little': 1, 'puke': 1, 'of': 1, 'these': 1, 'days': 1, 'catch': 1, 'and': 1, 'carve': 1, 'back': 1, 'with': 1, 'an': 1, 'ice': 1, 'pick': 1, 'whats': 1, 'matter': 1, "you're": 1, 'normal': 1, 'effervescent': 1, 'self': 1, 'homer_simpson': 1, 'i': 1, 'got': 1, 'moe': 1, 'give': 1, 'me': 1, 'another': 1, 'forget': 1, 'barney_gumble': 1, 'only': 1, 'enhance': 1, 'social': 1, 'skills': 1})
['moe_szyslak', 'mike', 'rotch', 'you', 'your', 'to', 'drink', 'the', 'yeah', 'name', 'on', 'hey', 'one', "i'm", 'gonna', 'my', 'homer', 'not', 'prob

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [8]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    lookup={
        '.':'||period||',
        ',':'||comma||',
        '"':'||quotation||',
        ';':'||semicolon||',
        '!':'||exclamation||',
        '?':'||question||',
        '(':'||left_paranthesis||',
        ')':'||right_paranthesis||',
        '--':'||dash||',
        '\n':'||return||'
    }
    return lookup

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

Counter({'||period||': 6024, '||return||': 4517, '||comma||': 3574, '||left_paranthesis||': 1550, '||right_paranthesis||': 1549, 'the': 1302, 'i': 1275, 'you': 1251, '||exclamation||': 1242, 'moe_szyslak:': 1180, '||question||': 1100, 'a': 1059, 'homer_simpson:': 975, 'to': 938, 'and': 631, 'of': 488, 'my': 474, 'it': 465, 'that': 441, 'in': 432, '||quotation||': 418, 'me': 412, 'is': 376, 'this': 373, "i'm": 334, 'for': 327, 'your': 324, 'homer': 307, 'on': 289, 'hey': 289, 'moe': 273, 'oh': 272, 'no': 261, 'lenny_leonard:': 247, 'what': 246, 'with': 235, 'yeah': 233, 'all': 231, 'just': 218, 'like': 214, 'but': 213, 'barney_gumble:': 207, 'so': 205, 'be': 204, 'here': 196, 'carl_carlson:': 194, "don't": 193, 'have': 192, 'up': 188, "it's": 188, 'well': 186, 'out': 185, 'do': 180, 'was': 179, 'got': 176, 'are': 175, 'get': 175, 'we': 172, 'uh': 168, "that's": 168, 'one': 164, "you're": 158, 'not': 156, 'now': 156, 'can': 155, 'know': 155, '||dash||': 151, 'at': 145, 'right': 144, '/':

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [2]:
print(token_dict)

{'.': '||period||', ',': '||comma||', '"': '||quotation||', ';': '||semicolon||', '!': '||exclamation||', '?': '||question||', '(': '||left_paranthesis||', ')': '||right_paranthesis||', '--': '||dash||', '\n': '||return||'}


## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [4]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Input=tf.placeholder(tf.int32,[None,None],name='input')
    Targets=tf.placeholder(tf.int32,[None,None],name='target')
    LearningRate=tf.placeholder(tf.float32,name='learn_rate')
    return (Input, Targets, LearningRate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [5]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm=tf.contrib.rnn.BasicLSTMCell(rnn_size)
    Cell=tf.contrib.rnn.MultiRNNCell([lstm])
    InitialState=Cell.zero_state(batch_size,tf.float32)
    InitialState=tf.identity(InitialState,'initial_state')
    return (Cell, InitialState)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [6]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding=tf.Variable(tf.random_uniform((vocab_size,embed_dim),-1,1))
    embed=tf.nn.embedding_lookup(embedding,input_data)
    return embed



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [7]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    Outputs,FinalState=tf.nn.dynamic_rnn(cell,inputs,dtype=tf.float32)
    FinalState=tf.identity(FinalState,'final_state')
    return (Outputs, FinalState)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [8]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed=get_embed(input_data, vocab_size, embed_dim)
    rnn,FinalState=build_rnn(cell, embed)
    print(rnn.shape)
    Logits=tf.contrib.layers.fully_connected(rnn,vocab_size,activation_fn=None,weights_initializer=tf.truncated_normal_initializer(stddev=0.1),biases_initializer=tf.zeros_initializer())
    print(Logits.shape)
    return (Logits, FinalState)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

(128, 5, 256)
(128, 5, 27)
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [9]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    starting=0
    x=batch_size*seq_length
    y=len(int_text)//x
    z=y*x
    text_x=int_text[:z]
    text_y=text_x[1:]+[text_x[0]]
    counter=0
    batch = np.zeros(shape = (y, 2, batch_size, seq_length), dtype = np.int32)
    for i in range (0,len(text_x),y*seq_length):
        seq_x=text_x[i:i+y*seq_length]
        seq_y=text_y[i:i+y*seq_length]
        
        count2=0
        for j in range(0,len(seq_x),seq_length):
            x_sample=seq_x[j:j+seq_length]
            y_sample=seq_y[j:j+seq_length]
            batch[count2,0,counter,:]=x_sample
            batch[count2,1,counter,:]=y_sample
            count2+=1
        counter+=1
    print(batch)
    return np.array(batch)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

[[[[   0    1    2    3    4]
   [  35   36   37   38   39]
   [  70   71   72   73   74]
   ..., 
   [4375 4376 4377 4378 4379]
   [4410 4411 4412 4413 4414]
   [4445 4446 4447 4448 4449]]

  [[   1    2    3    4    5]
   [  36   37   38   39   40]
   [  71   72   73   74   75]
   ..., 
   [4376 4377 4378 4379 4380]
   [4411 4412 4413 4414 4415]
   [4446 4447 4448 4449 4450]]]


 [[[   5    6    7    8    9]
   [  40   41   42   43   44]
   [  75   76   77   78   79]
   ..., 
   [4380 4381 4382 4383 4384]
   [4415 4416 4417 4418 4419]
   [4450 4451 4452 4453 4454]]

  [[   6    7    8    9   10]
   [  41   42   43   44   45]
   [  76   77   78   79   80]
   ..., 
   [4381 4382 4383 4384 4385]
   [4416 4417 4418 4419 4420]
   [4451 4452 4453 4454 4455]]]


 [[[  10   11   12   13   14]
   [  45   46   47   48   49]
   [  80   81   82   83   84]
   ..., 
   [4385 4386 4387 4388 4389]
   [4420 4421 4422 4423 4424]
   [4455 4456 4457 4458 4459]]

  [[  11   12   13   14   15]
   [  46   

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [10]:
# Number of Epochs
num_epochs = 150
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 20

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

(?, ?, 512)
(?, ?, 6779)


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

[[[[   9    3  121 ...,   13  145    0]
   [   0  161   21 ...,    7  186   62]
   [ 230   75  398 ...,    0    0    0]
   ..., 
   [  47  454   76 ...,    0    1  239]
   [  63    7  451 ...,    1   12    3]
   [ 291    8    1 ...,   65   49    7]]

  [[   3  121  172 ...,  145    0    1]
   [ 161   21  233 ...,  186   62  145]
   [  75  398   10 ...,    0    0    1]
   ..., 
   [ 454   76   11 ...,    1  239    6]
   [   7  451 6326 ...,   12    3 6327]
   [   8    1   45 ...,   49    7    0]]]


 [[[   1  146  174 ...,  203  198    2]
   [ 145   13  470 ...,  152  145   13]
   [   1 2903    3 ...,   39   23   22]
   ..., 
   [   6  156  511 ...,  124 1436    0]
   [6327    4 1186 ...,   47  440   35]
   [   0    1   45 ...,   58    2  212]]

  [[ 146  174    2 ...,  198    2 1170]
   [  13  470   26 ...,  145   13 2897]
   [2903    3 2904 ...,   23   22    5]
   ..., 
   [ 156  511   11 ..., 1436    0    0]
   [   4 1186  801 ...,  440   35    7]
   [   1   45    3 ...,    2  212   

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [15]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor=loaded_graph.get_tensor_by_name(name='input:0')
    InitialStateTensor=loaded_graph.get_tensor_by_name(name='initial_state:0')
    FinalStateTensor=loaded_graph.get_tensor_by_name(name='final_state:0')
    ProbsTensor=loaded_graph.get_tensor_by_name(name='probs:0')
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [16]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
   
    id=np.random.choice(probabilities)
    y=int_to_vocab[list(probabilities).index(id)]
   
    return y


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [17]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: whoa mansions ralph_wiggum: hungry drunkenly bam past waitress dyin' desire thankful mulder asked fat_in_the_hat: stools i/you kay discriminate founded bush guiltily wildest contact hall curds what'll lisa_simpson: been polygon edison collette: forgot department principles phony seething glen un-sults giving craphole men: aren'tcha sunk botanical new_health_inspector: means sometime lovely angrily costume's book ron_howard: mull teenage_barney: bless where'd kinderhook chuckles beeps clenched laughs referee naively plastered racially-diverse collector's specials pancakes windowshade aisle noosey tomorrow's she's local sagacity motor moans win murmur bless consider savings glum initially maitre whoo dads startin' winded sidelines f-l-a-n-r-d-s triangle dirge-like duff fritz consoling lifestyle unhappy aisle terrorizing rage distinct behind diving dawning almond cheesecake cotton jail timbuk-tee stole fl o evils gangrene drape

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.